# fold-rank

## 라이브러리 설치

In [ ]:
# 런타임 다시 시작
!pip install catboost

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 라이브러리 import

In [ ]:
# 필요한 라이브러리 불러오기
import pandas as pd
import numpy as np
import random
import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from scipy.stats import rankdata
from google.colab import files

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 라이브러리 버전
'''
Python version: 3.11.11
pandas version: 2.2.2
numpy version: 1.26.4
lightgbm version: 4.5.0
catboost version: 1.2.7
xgboost version: 2.1.4
scikit-learn version: 1.6.1
matplotlib version: 3.10.0
seaborn version: 0.13.2
scipy version: 1.13.1
'''

In [ ]:
# seed 고정
sd=42
random.seed(sd)
np.random.seed(sd)
os.environ['PYTHONHASHSEED'] = str(sd)

## 데이터 preprocessing

In [ ]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [ ]:
# ID 분리
test_ids = test_df['ID']  # ID 컬럼 따로 저장
train_df.drop(columns=['ID'], inplace=True)
test_df.drop(columns=['ID'], inplace=True)

In [ ]:
# 난자 출처 알 수 없음 -> 본인제공 replace
train_df['난자 출처'] = train_df['난자 출처'].replace('알 수 없음', '본인 제공')
test_df['난자 출처'] = test_df['난자 출처'].replace('알 수 없음', '본인 제공')

In [ ]:
# Unknown
train_df['특정 시술 유형'] = train_df['특정 시술 유형'].fillna('Unknown')
test_df['특정 시술 유형'] = test_df['특정 시술 유형'].fillna('Unknown')

# 특정 컬럼 먼저 대체
s_col = ['PGD 시술 여부', '착상 전 유전 검사 사용 여부', 'PGS 시술 여부']
train_df[s_col] = train_df[s_col].fillna(0)
test_df[s_col] = test_df[s_col].fillna(0)

# IVF가 아닌 DI의 경우 결측치 대체
# 설문 조사에서 DI의 경우 대답할 필요가 없는 항목들이 결측치로 들어간 듯
columns_to_update = ['단일 배아 이식 여부', '착상 전 유전 진단 사용 여부', '배아 생성 주요 이유',
                     '총 생성 배아 수', '미세주입된 난자 수', '미세주입에서 생성된 배아 수',
                     '이식된 배아 수', '미세주입 배아 이식 수', '저장된 배아 수','미세주입 후 저장된 배아 수',
                     '해동된 배아 수', '해동 난자 수', '수집된 신선 난자 수', '저장된 신선 난자 수',
                     '혼합된 난자 수', '파트너 정자와 혼합된 난자 수', '기증자 정자와 혼합된 난자 수',
                     '동결 배아 사용 여부', '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부']

# train_df 처리
for column in columns_to_update:
    if train_df[column].dtype == 'object':
        # object 타입이면 'Not Answer(DI)'로 채우기
        train_df[column] = train_df[column].fillna('Not Answer(DI)')
    elif train_df[column].dtype in ['float64', 'int64']:
        # 숫자 타입이면 0로 채우기
        train_df[column] = train_df[column].fillna(0)

# test_df 처리
for column in columns_to_update:
    if test_df[column].dtype == 'object':
        # object 타입이면 'Not Answer(DI)'로 채우기
        test_df[column] = test_df[column].fillna('Not Answer(DI)')
    elif test_df[column].dtype in ['float64', 'int64']:
        # 숫자 타입이면 0로 채우기
        test_df[column] = test_df[column].fillna(0)

# 경과일 컬럼 대체
o_col = ['난자 채취 경과일']
train_df[o_col] = train_df[o_col].fillna(1)
test_df[o_col] = test_df[o_col].fillna(1)

d_col = ['난자 혼합 경과일', '배아 이식 경과일', '배아 해동 경과일']
train_df[d_col] = train_df[d_col].fillna(999)
test_df[d_col] = test_df[d_col].fillna(999)

In [ ]:
# 횟수, 회 타입 변경
int_col = ['총 시술 횟수','클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수',
           '총 임신 횟수', 'IVF 임신 횟수','DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수',
           'DI 출산 횟수']

for col in int_col:
    train_df[col] = train_df[col].astype(str).str.extract('(\d+)').astype(int)
    test_df[col] = test_df[col].astype(str).str.extract('(\d+)').astype(int)

In [ ]:
# 값이 1인 항목의 이름 추출
to_drop = train_df['배란 유도 유형'].value_counts()[train_df['배란 유도 유형'].value_counts() == 1].index

# 해당 항목 제거 (train_df만)
train_df = train_df[~train_df['배란 유도 유형'].isin(to_drop)]

In [ ]:
# 조건 : 난자 출처 == "본인 제공" → 난자 기증자 나이를 시술 당시 나이로 설정
condition2 = (train_df['난자 출처'] == "본인 제공")
train_df.loc[condition2, '난자 기증자 나이'] = train_df.loc[condition2, '시술 당시 나이']

condition2_t = (test_df['난자 출처'] == "본인 제공")
test_df.loc[condition2_t, '난자 기증자 나이'] = test_df.loc[condition2_t, '시술 당시 나이']

In [ ]:
# 컬럼 drop
drop_col = ['임신 시도 또는 마지막 임신 경과 연수', '난자 해동 경과일', '불임 원인 - 정자 형태', '불임 원인 - 정자 운동성',
            '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 농도', '불임 원인 - 자궁경부 문제', '불임 원인 - 여성 요인',
            '대리모 여부', '부부 부 불임 원인']


train_df.drop(drop_col, axis=1, inplace=True)
test_df.drop(drop_col, axis=1, inplace=True)

## 키워드 추출

In [ ]:
# 키워드 리스트 정의
keywords = ['기증용', '현재 시술용', '난자 저장용', '배아 저장용']

# 각 키워드에 대해 새로운 컬럼 생성
for keyword in keywords:
    train_df[keyword] = train_df['배아 생성 주요 이유'].str.contains(keyword).astype(int)
    test_df[keyword] = test_df['배아 생성 주요 이유'].str.contains(keyword).astype(int)

# 기존 col drop
train_df.drop('배아 생성 주요 이유', axis=1, inplace=True)
test_df.drop('배아 생성 주요 이유', axis=1, inplace=True)

In [ ]:
# 키워드 리스트 정의
unique_types = ['ICSI', 'IVF', 'Unknown', 'IUI',
                'BLASTOCYST', 'AH']# 'ICI', 'GenericDI', 'IVI'

# 각 키워드에 대해 새로운 컬럼 생성
for keyword in unique_types:
    train_df[keyword] = train_df['특정 시술 유형'].str.count(keyword)
    test_df[keyword] = test_df['특정 시술 유형'].str.count(keyword)

# 기존 col drop
train_df.drop('특정 시술 유형', axis=1, inplace=True)
test_df.drop('특정 시술 유형', axis=1, inplace=True)

## 파생변수

In [ ]:
# 시술 받았는데 임신을 못한 경우 = 난임
train_df['난임 여부'] = train_df['총 시술 횟수'] - train_df['총 임신 횟수']
test_df['난임 여부'] = test_df['총 시술 횟수'] - test_df['총 임신 횟수']

# 임신을 했는데 출산을 못한경우 = 유산
train_df['유산 여부'] = train_df['총 임신 횟수'] - train_df['총 출산 횟수']
train_df['유산 여부'] = train_df['유산 여부'].apply(lambda x: 1 if x > 0 else 0)

test_df['유산 여부'] = test_df['총 임신 횟수'] - test_df['총 출산 횟수']
test_df['유산 여부'] = test_df['유산 여부'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
# 미세주입(ICSI) 아닌 IVF
train_df['미세주입이 아닌 배아 이식 수'] = train_df['이식된 배아 수'] - train_df['미세주입 배아 이식 수']
test_df['미세주입이 아닌 배아 이식 수'] = test_df['이식된 배아 수'] - test_df['미세주입 배아 이식 수']

train_df['미세주입이 아닌 배아 생성 수'] = train_df['총 생성 배아 수'] - train_df['미세주입에서 생성된 배아 수']
test_df['미세주입이 아닌 배아 생성 수'] = test_df['총 생성 배아 수'] - test_df['미세주입에서 생성된 배아 수']

train_df['미세주입이 아닌 배아 저장 수'] = train_df['저장된 배아 수'] - train_df['미세주입 후 저장된 배아 수']
test_df['미세주입이 아닌 배아 저장 수'] = test_df['저장된 배아 수'] - test_df['미세주입 후 저장된 배아 수']

In [ ]:
train_df['클리닉 외 총 시술 횟수'] = train_df['총 시술 횟수'] - train_df['클리닉 내 총 시술 횟수']
test_df['클리닉 외 총 시술 횟수'] = test_df['총 시술 횟수'] - test_df['클리닉 내 총 시술 횟수']

train_df['PGS 검사를 받고도 안한 사람'] = train_df['착상 전 유전 검사 사용 여부'] - train_df['PGS 시술 여부']
test_df['PGS 검사를 받고도 안한 사람'] = test_df['착상 전 유전 검사 사용 여부'] - test_df['PGS 시술 여부']

## 배아 변수

In [ ]:
# 단일 배아를 이식한 경우의 이식된 배아 수가 1이면 성공률이 더 높음
train_df.loc[(train_df['단일 배아 이식 여부'] == 1) & (train_df['이식된 배아 수'] == 1), '이식된 배아 수'] = 1.5
test_df.loc[(test_df['단일 배아 이식 여부'] == 1) & (test_df['이식된 배아 수'] == 1), '이식된 배아 수'] = 1.5

train_df.loc[(train_df['착상 전 유전 검사 사용 여부'] == 1) & (train_df['PGS 시술 여부'] == 0), 'PGS 시술 여부'] = -1
test_df.loc[(test_df['착상 전 유전 검사 사용 여부'] == 1) & (test_df['PGS 시술 여부'] == 0), 'PGS 시술 여부'] = -1

In [ ]:
# 이식 배아 수/ 나이
# 나이 재 정의
age_bin_mapping = {
    "만18-34세": 1,
    "만35-37세": 2,
    "만38-39세": 3,
    "만40-42세": 4,
    "만43-44세": 5,
    "만45-50세": 6,
    "알 수 없음": 7
}

# 나이  적용
train_df['나이'] = train_df['시술 당시 나이'].map(age_bin_mapping)
test_df['나이'] = test_df['시술 당시 나이'].map(age_bin_mapping)

# 나이 별 배아수
train_df['이식 배아 수/나이'] = train_df['이식된 배아 수'] / train_df['나이']

test_df['이식 배아 수/나이'] = test_df['이식된 배아 수'] / test_df['나이']

# 나이 drop
train_df.drop('나이', axis=1, inplace=True)
test_df.drop('나이', axis=1, inplace=True)

## 범주형

In [ ]:
# categorical로 바꿔줌
cat_col = ['배란 자극 여부', '단일 배아 이식 여부', '착상 전 유전 검사 사용 여부', '착상 전 유전 진단 사용 여부',
 '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', '여성 부 불임 원인', '부부 주 불임 원인',
  '불명확 불임 원인', '불임 원인 - 난관 질환', '불임 원인 - 남성 요인', '불임 원인 - 배란 장애',
 '불임 원인 - 자궁내막증', '총 시술 횟수', '클리닉 내 총 시술 횟수', 'IVF 시술 횟수', 'DI 시술 횟수', '총 임신 횟수',
 'IVF 임신 횟수', 'DI 임신 횟수', '총 출산 횟수', 'IVF 출산 횟수', 'DI 출산 횟수', '동결 배아 사용 여부',
 '신선 배아 사용 여부', '기증 배아 사용 여부',  'PGD 시술 여부', 'PGS 시술 여부', '기증용', '현재 시술용',
 '난자 저장용', '배아 저장용', 'ICSI', 'IVF', 'Unknown', 'IUI', 'BLASTOCYST', 'AH', '난임 여부', '유산 여부',
 'PGS 검사를 받고도 안한 사람']

for col in cat_col:
    train_df[col] = train_df[col].astype(str)
    test_df[col] = test_df[col].astype(str)

In [ ]:
# 범주형 변수 지정
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# 범주형 변수를 category 타입으로 변환
for col in categorical_features:
    train_df[col] = train_df[col].astype("category")
    test_df[col] = test_df[col].astype("category")  # 테스트 데이터도 변환

## 모델링

In [ ]:
# 타겟 분리
y_train = train_df['임신 성공 여부']
X_train = train_df.drop(columns=['임신 성공 여부'])
X_test = test_df.copy()

## 클래스 가중치 계산
class_counts = y_train.value_counts()
pos_weight = class_counts[0] / class_counts[1]

## 🎯 K-Fold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=sd)

## 🎯 LGBM & CatBoost 모델 학습
lgb_params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'num_leaves': 31,
    'random_state': sd
}

cat_params = {
    'loss_function': 'Logloss',
    'eval_metric': 'AUC',
    'learning_rate': 0.1,
    'iterations': 1000,
    'depth': 6,
    'random_seed': sd,
    'od_wait': 100,
    'task_type': 'CPU',
    'verbose': 200,
    'scale_pos_weight': pos_weight  # 클래스 불균형 조정
}

xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',  # 평가 지표 AUC 설정
    'learning_rate': 0.1,
    'max_depth': 6,
    'n_estimators': 1000,
    'random_state': sd,
    'scale_pos_weight': pos_weight
}

## 🎯 모델별 AUC 저장 구조 변경
auc_scores_dict = {'lgbm': [], 'xgb': [], 'cat': []}
models_dict = {'lgbm': [], 'xgb': [], 'cat': []}
test_pred_dict = {'lgbm': [], 'xgb': [], 'cat': []}

In [ ]:
# 각 모델에서 드롭할 컬럼 지정
# LGBM에서는 drop하지 않는다
drop_features = {
    'lgbm': [],
    'xgb': ['PGS 검사를 받고도 안한 사람'],
    'cat': ['PGS 검사를 받고도 안한 사람']
}

# 모델별 데이터 준비
def prepare_model_data(model_name, X_train, X_test):
    X_train_model = X_train.drop(columns=drop_features[model_name])
    X_test_model = X_test.drop(columns=drop_features[model_name])
    categorical_features_model = [col for col in categorical_features if col not in drop_features[model_name]]
    return X_train_model, X_test_model, categorical_features_model

# LGBM 데이터 준비
X_train_lgbm, X_test_lgbm, categorical_features_lgbm = prepare_model_data('lgbm', X_train, X_test)
# XGBoost 데이터 준비
X_train_xgb, X_test_xgb, categorical_features_xgb = prepare_model_data('xgb', X_train, X_test)
# CatBoost 데이터 준비
X_train_cat, X_test_cat, categorical_features_cat = prepare_model_data('cat', X_train, X_test)

In [ ]:
### ✅ **5-Fold LGBM 학습**
for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_lgbm, y_train)):
    X_tr, X_val = X_train_lgbm.iloc[train_idx], X_train_lgbm.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

    # LGBM 데이터셋 생성
    lgb_train = lgb.Dataset(X_tr, label=y_tr, weight=y_tr.map({0: 1, 1: pos_weight}), categorical_feature=categorical_features_lgbm)
    lgb_valid = lgb.Dataset(X_val, label=y_val, weight=y_val.map({0: 1, 1: pos_weight}), categorical_feature=categorical_features_lgbm)

    # LGBM 학습
    model = lgb.train(lgb_params, lgb_train, valid_sets=[lgb_valid], callbacks=[early_stopping(100), log_evaluation(100)])
    models_dict['lgbm'].append(model)

    # 예측 수행
    y_pred_proba = model.predict(X_val)
    auc_score = roc_auc_score(y_val, y_pred_proba)
    auc_scores_dict['lgbm'].append((auc_score, fold))

    # 테스트 데이터 예측 저장
    test_pred_dict['lgbm'].append(model.predict(X_test_lgbm))

print(f"LGBM 완료! 평균 AUC: {np.mean([x[0] for x in auc_scores_dict['lgbm']]):.5f}")

In [ ]:
### ✅ **5-Fold XGBoost 학습 (DMatrix에서 범주형 변수 직접 사용)**
for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_xgb, y_train)):
    X_tr, X_val = X_train_xgb.iloc[train_idx], X_train_xgb.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

    # ✅ DMatrix 생성 (enable_categorical=True)
    dtrain = xgb.DMatrix(X_tr, label=y_tr, enable_categorical=True)
    dvalid = xgb.DMatrix(X_val, label=y_val, enable_categorical=True)

    # ✅ XGBoost 모델 학습
    model = xgb.train(
        xgb_params,
        dtrain,
        num_boost_round=1000,
        evals=[(dvalid, 'validation')],
        early_stopping_rounds=100,
        verbose_eval=100
    )

    models_dict['xgb'].append(model)

    # 예측 수행
    y_pred_proba = model.predict(xgb.DMatrix(X_val, enable_categorical=True))
    auc_score = roc_auc_score(y_val, y_pred_proba)
    auc_scores_dict['xgb'].append((auc_score, fold))

    # 테스트 데이터 예측 저장
    test_pred_dict['xgb'].append(model.predict(xgb.DMatrix(X_test_xgb, enable_categorical=True)))

print(f"XGBoost 완료! 평균 AUC: {np.mean([x[0] for x in auc_scores_dict['xgb']]):.5f}")

In [ ]:
### ✅ **5-Fold CatBoost 학습**
for fold, (train_idx, valid_idx) in enumerate(skf.split(X_train_cat, y_train)):
    X_tr, X_val = X_train_cat.iloc[train_idx], X_train_cat.iloc[valid_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[valid_idx]

    train_pool = Pool(X_tr, label=y_tr, cat_features=categorical_features_cat)
    valid_pool = Pool(X_val, label=y_val, cat_features=categorical_features_cat)

    # CatBoost 학습
    model = CatBoostClassifier(**cat_params)
    model.fit(train_pool, eval_set=valid_pool, early_stopping_rounds=100, verbose=100)

    models_dict['cat'].append(model)

    # 예측 수행
    y_pred_proba = model.predict_proba(X_val)[:, 1]
    auc_score = roc_auc_score(y_val, y_pred_proba)
    auc_scores_dict['cat'].append((auc_score, fold))

    # 테스트 데이터 예측 저장
    test_pred_dict['cat'].append(model.predict_proba(X_test_cat)[:, 1])

print(f"CatBoost 완료! 평균 AUC: {np.mean([x[0] for x in auc_scores_dict['cat']]):.5f}")

## 최종 학습 제출

In [ ]:
## 🎯 **가중 랭크 변환을 이용한 앙상블**
def weighted_rank_ensemble(predictions, weights):
    """ 가중 랭크 변환을 적용한 앙상블 """
    predictions = np.array(predictions)
    ranked_preds = np.array([rankdata(pred) for pred in predictions])
    weighted_avg_rank = np.average(ranked_preds, axis=0, weights=weights)
    return weighted_avg_rank / np.max(weighted_avg_rank)  # 정규화

## 🎯 가중치 설정
manual_weights = {'lgbm': 1.3, 'xgb': 3, 'cat': 2.9}

## 🎯 n=3일 때 가중 랭크 앙상블 수행
n = 3
print(f"\n🔹 현재 n 값: {n}")

selected_test_preds = []
selected_weights = []

for model_type in ['lgbm', 'xgb', 'cat']:
    # 모델별로 AUC 높은 n개 선택
    top_n_folds = sorted(auc_scores_dict[model_type], reverse=True, key=lambda x: x[0])[:n]
    top_n_folds_idx = [fold_idx for _, fold_idx in top_n_folds]

    print(f"{model_type} 선택된 fold: {top_n_folds_idx}")

    # 선택된 fold의 예측값 저장
    selected_test_preds += [test_pred_dict[model_type][idx] for idx in top_n_folds_idx]

    # 선택된 fold 개수만큼 가중치 추가
    selected_weights += [manual_weights[model_type]] * len(top_n_folds_idx)

# 가중 랭크 앙상블 수행
y_test_proba = weighted_rank_ensemble(selected_test_preds, selected_weights)

# 제출 파일 생성
file_name = f'ensemble-weighted-rank-top{n}-per-model-fold.csv'
submission_df = pd.read_csv("./data/sample_submission.csv")
submission_df['ID'] = test_ids
submission_df.iloc[:, 1] = y_test_proba
submission_df.to_csv(file_name, index=False)

# 파일 다운로드
files.download(file_name)



🔹 현재 n 값: 3
lgbm 선택된 fold: [1, 2, 3]
xgb 선택된 fold: [1, 3, 2]
cat 선택된 fold: [1, 2, 3]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>